## Import 

In [14]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.object_detection import non_max_suppression
import numpy as np
import pickle
import argparse
import cv2

## Selective Search Region Proposals
- Define the maximum number of Selective Search region proposals to be utilized for training and inference

In [5]:
# define the number of max proposals used when running selective
# search for (1) gathering training data and (2) performing inference
max_proposals = 2000
max_proposals_infer = 200

## Max Number of Positive and Negative Regions

In [6]:
# define the maximum number of positive and negative images to be
# generated from each image
max_pos = 30
max_neg = 10

## Initializing Dimension Input
- i.e. Image input

In [7]:
# initialize the input dimensions to the network
image_input = (300, 300)

# define the path to the output model
model_path = "model_3.hdf5"

# define the minimum probability required for a positive prediction
# (used to filter out false-positive predictions)
min_prob = 0.80

In [9]:
# CNN model path
model_path = "model_3.hdf5"

In [17]:
# load the our fine-tuned model and label binarizer from disk
print("[INFO] loading model...")
model = load_model(model_path)

# load the input image from disk
image = cv2.imread("Data-Images")
image = imutils.resize(image, width=500)

# run selective search on the image to generate bounding box proposal
# regions
# print("[INFO] running selective search...")
# ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
# ss.setBaseImage(image)
# ss.switchToSelectiveSearchFast()
# rects = ss.process()

[INFO] loading model...


NameError: name 'imutils' is not defined